# Lab 5: Error mitigation with Qiskit Runtime

In this lab, we'll explore a few of the error mitigation options available through Qiskit Runtime. Specifically, we'll define a simple observable and initial state and use the Estimator primitive to measure the expectation value. Using noisy simulations, we'll explore the effect of different error mitigation strategies.

## Setup

We'll define a simple Heisenberg Hamiltonian model to use as an example. We'll also construct a simple state preparation circuit.

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.quantum_info import SparsePauliOp


def heisenberg_hamiltonian(
    length: int, jx: float = 1.0, jy: float = 0.0, jz: float = 0.0
) -> SparsePauliOp:
    terms = []
    for i in range(length - 1):
        if jx:
            terms.append(("XX", [i, i + 1], jx))
        if jy:
            terms.append(("YY", [i, i + 1], jy))
        if jz:
            terms.append(("ZZ", [i, i + 1], jz))
    return SparsePauliOp.from_sparse_list(terms, num_qubits=length)


def state_prep_circuit(num_qubits: int, layers: int = 1) -> QuantumCircuit:
    qubits = QuantumRegister(num_qubits, name="q")
    circuit = QuantumCircuit(qubits)
    circuit.h(qubits)
    for _ in range(layers):
        for i in range(0, num_qubits - 1, 2):
            circuit.cx(qubits[i], qubits[i + 1])
        circuit.ry(0.1, qubits)
        for i in range(1, num_qubits - 1, 2):
            circuit.cx(qubits[i], qubits[i + 1])
        circuit.ry(0.1, qubits)
    return circuit

In [ ]:
length = 5

hamiltonian = heisenberg_hamiltonian(length, 1.0, 1.0)
circuit = state_prep_circuit(length, layers=2)

print(hamiltonian)
circuit.draw("mpl")

## Calculate exact expectation value (energy)

First, we'll calculate the exact expectation value using a local simulator implementation of the Estimator primitive. The expectation value of a Hamiltonian is also referred to as "energy."

In [ ]:
from qiskit_aer.primitives import Estimator

estimator = Estimator(approximation=True)
job = estimator.run(circuit, hamiltonian, shots=None)
result = job.result()
exact_value = result.values[0]

print(f"Exact energy: {exact_value}")

## Run noisy simulation through Qiskit Runtime

Next, we'll initialize the Qiskit Runtime service and switch to using its Estimator primitive, backed by a simulator that can handle noise. Even though our circuit acts on 5 qubits, we'll initialize a simulator with 6 qubits in order to later demonstrate the potential effects of qubit choice.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

hub = "ibm-q-internal"
group = "deployed"
project = "default"
service = QiskitRuntimeService(instance=f"{hub}/{group}/{project}")

In [ ]:
from qiskit_ibm_runtime import Estimator, Options, Session
from qiskit.transpiler import CouplingMap

backend = service.get_backend("simulator_statevector")
# set simulation options
simulator = {
    "basis_gates": ["id", "rz", "sx", "cx", "reset"],
    "coupling_map": list(CouplingMap.from_line(length + 1)),
}
shots = 10000

### No noise

First, we'll run the simulation with no noise.

In [ ]:
import math

options = Options(
    simulator=simulator,
    resilience_level=0,
)

with Session(service=service, backend=backend):
    estimator = Estimator(options=options)
    job = estimator.run(circuit, hamiltonian, shots=shots)

result = job.result()
experiment_value = result.values[0]
error = abs(experiment_value - exact_value)
variance = result.metadata[0]["variance"]
std = math.sqrt(variance / shots)

print(f"Estimated energy: {experiment_value}")
print(f"Energy error: {error}")
print(f"Variance: {variance}")
print(f"Standard error: {std}")

### Readout error

Next, let's run a simulation with readout error.

#### Exercise 1

In this exercise, you'll construct a noise model that has modest readout error on all qubits except for the first qubit, which will have really bad readout error.

Specifically, construct a noise model with the following properties:
- For the first qubit (qubit 0):
  - A readout of 1 has a 50% probability of being erroneously read as 0.
  - A readout of 0 has a 20% probability of being erroneously read as 1.
- For the rest of the qubits:
  - A readout of 1 has a 5% probability of being erroneously read as 0.
  - A readout of 0 has a 2% probability of being erroneously read as 1.

You may find it helpful to consult the following resources:
  - https://qiskit.org/ecosystem/aer/apidocs/aer_noise.html
  - https://qiskit.org/documentation/tutorials/simulators/3_building_noise_models.html

In [ ]:
from qiskit_aer.noise import NoiseModel, ReadoutError

noise_model = NoiseModel()

##### your code here #####

print(noise_model)

In [ ]:
# Submit your answer

from qc_grader.challenges.qgss_2023 import grade_lab5_ex1

grade_lab5_ex1(noise_model)

First, let's try running the simulation without doing anything to mitigate the readout error. We'll explicitly set `resilience_level = 0` to ensure that no error mitigation is applied by the Runtime service. To illustrate the effect of a poor choice of qubits, we'll explicitly specify an initial layout that includes qubit 0.

In [ ]:
options = Options(
    simulator=dict(noise_model=noise_model, **simulator),
    resilience_level=0,
    transpilation=dict(initial_layout=list(range(length))),
)

with Session(service=service, backend=backend):
    estimator = Estimator(options=options)
    job = estimator.run(circuit, hamiltonian, shots=shots)

result = job.result()
experiment_value = result.values[0]
error = abs(experiment_value - exact_value)
variance = result.metadata[0]["variance"]
std = math.sqrt(variance / shots)

print(f"Estimated energy: {experiment_value}")
print(f"Energy error: {error}")
print(f"Variance: {variance}")
print(f"Standard error: {std}")

The error we get is pretty large. To improve things, let's pick a qubit layout that avoids qubit 0.

In [ ]:
options = Options(
    simulator=dict(noise_model=noise_model, **simulator),
    resilience_level=0,
    transpilation=dict(initial_layout=list(range(1, length + 1))),
)

with Session(service=service, backend=backend):
    estimator = Estimator(options=options)
    job = estimator.run(circuit, hamiltonian, shots=shots)

result = job.result()
experiment_value = result.values[0]
error = abs(experiment_value - exact_value)
variance = result.metadata[0]["variance"]
std = math.sqrt(variance / shots)

print(f"Estimated energy: {experiment_value}")
print(f"Energy error: {error}")
print(f"Variance: {variance}")
print(f"Standard error: {std}")

The error is smaller now, but still significant. Let's now enable readout error mitigation by setting `resilience_level = 1`.

In [ ]:
options = Options(
    simulator=dict(noise_model=noise_model, **simulator),
    resilience_level=1,
    transpilation=dict(initial_layout=list(range(1, length + 1))),
)

with Session(service=service, backend=backend):
    estimator = Estimator(options=options)
    job = estimator.run(circuit, hamiltonian, shots=shots)

result = job.result()
experiment_value = result.values[0]
error = abs(experiment_value - exact_value)
variance = result.metadata[0]["variance"]
std = math.sqrt(variance / shots)

print(f"Estimated energy: {experiment_value}")
print(f"Energy error: {error}")
print(f"Variance: {variance}")
print(f"Standard error: {std}")

Now, the effect of readout error has been almost completely mitigated! This mitigation did not come for free. In particular,
- To perform readout error mitigation, the Runtime service has to run additional calibration circuits, so the overall running time may be longer.
- The variance of the estimator has increased, leading to a larger standard error of the mean. As a consequence, a larger number of shots needs to be specified in order to achieve a given standard error.

Typically, these costs are relatively small, so it is almost always worthwhile to enable readout error mitigation.

#### Exercise 2

Suppose that turning on readout error mitigation increases the variance of your estimator by a factor of 2. If you originally ran your experiment with 10,000 shots, how many shots should you now use to achieve the same standard error of the mean?

In [ ]:
new_shots: int

##### your code here #####


In [ ]:
# Submit your answer

from qc_grader.challenges.qgss_2023 import grade_lab5_ex2

grade_lab5_ex2(new_shots)

### Depolarizing error and zero-noise extrapolation

In this section, we will see how depolarizing error can be mitigated using zero-noise extrapolation. Because the zero-noise extrapolation feature of Qiskit Runtime is still in beta, it currently has a few limitations. In particular, as of the time of this writing, the zero-noise extrapolation feature does not mitigate readout error. Therefore, in the examples below, we will remove readout error from our noise model.

#### Exercise 3

Construct a noise model that adds two-qubit depolarizing error after each CNOT gate, such that the error channel maps the input quantum state to the completely mixed state with 1% probability.

In [ ]:
from qiskit_aer.noise import depolarizing_error

noise_model = NoiseModel()

##### your code here #####

print(noise_model)

In [ ]:
# Submit your answer

from qc_grader.challenges.qgss_2023 import grade_lab5_ex3

grade_lab5_ex3(noise_model)

Let's run the estimator with `resilience_level = 1`, which turns on readout error mitigation. Because our noise model doesn't include readout error mitigation, we don't expect this to help.

In [ ]:
options = Options(
    simulator=dict(noise_model=noise_model, **simulator),
    resilience_level=1,
)

with Session(service=service, backend=backend):
    estimator = Estimator(options=options)
    job = estimator.run(circuit, hamiltonian, shots=shots)

result = job.result()
experiment_value = result.values[0]
error = abs(experiment_value - exact_value)
variance = result.metadata[0]["variance"]
std = math.sqrt(variance / shots)

print(f"Estimated energy: {experiment_value}")
print(f"Energy error: {error}")
print(f"Variance: {variance}")
print(f"Standard error: {std}")

As expected, the error we get is pretty significant.

Now, let's turn on zero-noise extrapolation by setting `resilience_level = 2`.

In [ ]:
options = Options(
    simulator=dict(noise_model=noise_model, **simulator),
    resilience_level=2,
)

with Session(service=service, backend=backend):
    estimator = Estimator(options=options)
    job = estimator.run(circuit, hamiltonian, shots=shots)

result = job.result()
experiment_value = result.values[0]
error = abs(experiment_value - exact_value)
variances = result.metadata[0]["zne"]["noise_amplification"]["variance"]

print(f"Estimated energy: {experiment_value}")
print(f"Energy error: {error}")
print(f"Variances: {variances}")

Now, the effect of depolarizing noise has been almost completely mitigated! Note that instead of getting a single variance value for the estimator, we are now returned a list of variances, one for each data point measured for the extrapolation. In a future version of Qiskit Runtime, these variances will also be extrapolated to return a single variance for the final estimator.

#### Exercise 4 (ungraded)

Besides depolarizing error, what other kinds of noise can be mitigated by zero-noise extrapolation? Test your proposals by constructing other noise models, and then simulating them with and without zero-noise extrapolation.